In [34]:
import torch as pt
from torch import nn
import torchaudio as ta
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader

import numpy as numpy

import os
import subprocess
import tqdm as tqdm
import json

In [30]:
if pt.cuda.is_available():
    device='cuda'
else:
    device='cpu'

In [31]:
print(device)

cuda


In [21]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        name,ext=os.path.splitext(file)
        new_name=name+'.wav'
        os.rename(os.path.join(path,file),os.path.join(path,new_name))

In [13]:
def convert_webm_to_wav(file):
    command = ['ffmpeg', '-i', file, file[:-5] + '.wav']
    subprocess.run(command)

In [19]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        subprocess.run(['ffmpeg','-i',os.path.join(path,file),'test_df/'+file[:-5]+'.wav'])

In [2]:
path='test_df'
name_set=set()
for file in os.listdir(path):
    if file.endswith('wav'):
        name_set.add(file)
print(len(name_set))

25984


In [24]:
t=os.path.join(path,list(name_set)[0])
label_path='data\public_dataset'
fname=t[8:-4]
l=os.path.join(label_path,fname+'.json')
print(fname)
with open(l,'r') as f:
    content=json.loads(f.read())
print(content)

2d71f678-6687-4a2f-bfed-e40fd23b9752
{'datetime': '2020-04-14T18:46:15.579555+00:00', 'cough_detected': '0.8779', 'latitude': '37.8', 'longitude': '29.1', 'age': '8', 'gender': 'male', 'respiratory_condition': 'False', 'fever_muscle_pain': 'False', 'status': 'healthy'}


In [27]:
print(list(name_set)[0])

2d71f678-6687-4a2f-bfed-e40fd23b9752.wav


In [6]:
signal,sr=ta.load(t)

In [16]:
print(content['cough_detected'])

0.9902


In [32]:
class CoughVidDataset(Dataset):

    def __init__(self,audio_path,label_path,transformation,target_sample_rate,num_samples,device):
        name_set=set()
        for file in os.listdir(audio_path):
            if file.endswith('wav'):
                name_set.add(file)
        name_set=list(name_set)
        self.datalist=name_set
        self.audio_path=audio_path
        self.label_path=label_path
        self.device=device
        self.transformation=transformation.to(device)
        self.target_sample_rate=target_sample_rate
        self.num_samples=num_samples
        
    def __len__(self):
        return len(self.datalist)

    def __getitem__(self,idx):
        audio_file_path=os.path.join(self.audio_path,self.datalist[idx])
        label_file_path=os.path.join(self.label_path,self.datalist[idx][:-4]+'.json')
        with open(label_file_path,'r') as f:
            content=json.loads(f.read())
            f.close()
        label=content['cough_detected']
        waveform,sample_rate=ta.load(audio_file_path) #(num_channels,samples) -> (1,samples) makes the waveform mono
        waveform=waveform.to(self.device)
        waveform=self._resample(waveform,sample_rate)   
        waveform=self._mix_down(waveform)
        waveform=self._cut(waveform)
        waveform=self._right_pad(waveform)
        waveform=self.transformation(waveform)
        return waveform,label

    def _resample(self,waveform,sample_rate):
        # used to handle sample rate
        resampler=ta.transforms.Resample(sample_rate,self.target_sample_rate)
        return resampler(waveform)
    
    def _mix_down(self,waveform):
        # used to handle channels
        waveform=pt.mean(waveform,dim=0,keepdim=True)
        return waveform
    
    def _cut(self,waveform):
        # cuts the waveform if it has more than certain samples
        if waveform.shape[1]>self.num_samples:
            waveform=waveform[:,:self.num_samples]
        return waveform
    
    def _right_pad(self,waveform):
        # pads the waveform if it has less than certain samples
        signal_length=waveform.shape[1]
        if signal_length<self.num_samples:
            num_padding=self.num_samples-signal_length
            last_dim_padding=(0,num_padding) # first arg for left second for right padding. Make a list of tuples for multi dim
            waveform=pt.nn.functional.pad(waveform,last_dim_padding)
        return waveform



In [35]:
class CNNNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3=nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4=nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten=nn.Flatten()
        self.linear1=nn.Linear(in_features=128*5*4,out_features=128)
        self.linear2=nn.Linear(in_features=128,out_features=1)
        self.output=nn.Sigmoid()
    
    def forward(self,input_data):
        x=self.conv1(input_data)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.flatten(x)
        x=self.linear1(x)
        logits=self.linear2(x)
        output=self.output(logits)
        
        return output


In [10]:
audio_path='test_df'
label_path='data\public_dataset'
SAMPLE_RATE=22050
NUM_SAMPLES=22050

In [28]:
melspectogram=ta.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE,n_fft=1024,hop_length=512,n_mels=64)
coughvid_dataset=CoughVidDataset(audio_path,label_path,melspectogram,SAMPLE_RATE,NUM_SAMPLES,device)

In [39]:
model=CNNNetwork().cuda()
summary(model,(1,64,44))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 33, 23]          --
|    └─Conv2d: 2-1                       [-1, 16, 66, 46]          160
|    └─ReLU: 2-2                         [-1, 16, 66, 46]          --
|    └─MaxPool2d: 2-3                    [-1, 16, 33, 23]          --
├─Sequential: 1-2                        [-1, 32, 17, 12]          --
|    └─Conv2d: 2-4                       [-1, 32, 35, 25]          4,640
|    └─ReLU: 2-5                         [-1, 32, 35, 25]          --
|    └─MaxPool2d: 2-6                    [-1, 32, 17, 12]          --
├─Sequential: 1-3                        [-1, 64, 9, 7]            --
|    └─Conv2d: 2-7                       [-1, 64, 19, 14]          18,496
|    └─ReLU: 2-8                         [-1, 64, 19, 14]          --
|    └─MaxPool2d: 2-9                    [-1, 64, 9, 7]            --
├─Sequential: 1-4                        [-1, 128, 5, 4]           --
|    └─

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 33, 23]          --
|    └─Conv2d: 2-1                       [-1, 16, 66, 46]          160
|    └─ReLU: 2-2                         [-1, 16, 66, 46]          --
|    └─MaxPool2d: 2-3                    [-1, 16, 33, 23]          --
├─Sequential: 1-2                        [-1, 32, 17, 12]          --
|    └─Conv2d: 2-4                       [-1, 32, 35, 25]          4,640
|    └─ReLU: 2-5                         [-1, 32, 35, 25]          --
|    └─MaxPool2d: 2-6                    [-1, 32, 17, 12]          --
├─Sequential: 1-3                        [-1, 64, 9, 7]            --
|    └─Conv2d: 2-7                       [-1, 64, 19, 14]          18,496
|    └─ReLU: 2-8                         [-1, 64, 19, 14]          --
|    └─MaxPool2d: 2-9                    [-1, 64, 9, 7]            --
├─Sequential: 1-4                        [-1, 128, 5, 4]           --
|    └─

In [29]:
signal,label=coughvid_dataset[0]
print(signal.shape)

torch.Size([1, 64, 44])
